In [168]:
import pandas as pd
import numpy as np

df = pd.read_stata("./serrano/serrano_2024_Stata/serrano.dta")

In [169]:
display(len(df))
display(df["ORGNR"].nunique())

15327519

1550192

# General Filters

In [304]:
"""
These filter should be thought through and written about in the methodology section.
We want to avoid cherry-picking the data, but we also want to avoid including data that is not relevant to the study.

We also want to avoid survuvalrship bias, i.e., we want to avoid including only the companies that have survived.
"""
# Maybe add something about negative equity, i.e., companies that are bankrupt or close to bankruptcy.
filtered_df = df[(df['ser_year'] >= 2011) & # 2012-2019 for final dataset, stable period with no major economic crises, here 2011 so we can get growth measures for 2012
                 (df['ser_year'] <= 2019) & # and stable interest rates, bnp growth, inflation
                 (df['ser_jurform'] == 49) & # aktiebolag
                 (df['ser_aktiv'] == 1) & # active companies
                 (df['ser_ftgkategori'] == 30) & # private companies, i.e., not state-owned etc.
                 (df['ser_stklf'] >= 2) & # companies with at least 10 employee for final dataset. Include smaller here to then drop
                 (df['knc_kncfall'] == 1) & # only include independet companies, i.e., not subsidiaries or parent companies THIS IS UP FOR DEBATE
                 (df['rr01_ntoms'] > 0) & # remove companies with no turnover
                 (df['ser_inregyr'] == 1)] # only include companies that are registered at SCB (just three observations are not)
# unsure about the last one, might remove to make the dataset larger.

In [305]:
display(len(filtered_df))
display(filtered_df["ORGNR"].nunique())

310859

72117

In [306]:
# Sort the dataframe so ORGNR are together and sorted by year after that.
filtered_df = filtered_df.sort_values(by=["ORGNR", "ser_year"])

In [307]:
filtered_df["ser_year"].value_counts()

ser_year
2011.0    35012
2018.0    34734
2016.0    34699
2012.0    34672
2017.0    34580
2019.0    34415
2013.0    34368
2015.0    34352
2014.0    34027
Name: count, dtype: int64

In [308]:
"""
Only include companies that fit all criteria in order, no year should be missing from the series.

Maybe do this.
"""

'\nOnly include companies that fit all criteria in order, no year should be missing from the series.\n\nMaybe do this.\n'

# Growth Variable

In [309]:
"""
Adding a variable for growth.

This is according to OECD's definition of high-growth firms,
which is defined as firms with an average annualized growth greater than 20%
per annum over a three-year period, and with ten or more employees at the beginning of the period.
"""

# Calculate the annual growth rate for each company and add it as a new column
filtered_df['TURNOVER_GROWTH'] = filtered_df.groupby('ORGNR')['rr01_ntoms'].pct_change()

# Identify high-growth periods and create a new binary column
filtered_df['HIGH_GROWTH'] = filtered_df.groupby('ORGNR').apply(
    lambda x: ((x['TURNOVER_GROWTH'] > 0.20).rolling(window=3).sum() == 3)
).reset_index(level=0, drop=True).astype(int)



/var/folders/pd/d0m27jy17d36rd2w_8y762cr0000gn/T/ipykernel_30763/3022460389.py:13: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  filtered_df['HIGH_GROWTH'] = filtered_df.groupby('ORGNR').apply(


In [310]:
"""
Drop 2011 observaitons. It was only included to calculate the growth rate for 2012.

Drop companies w/ fewer than 10 employees. They were included to calculate the growth for companies that are included in the final dataset.
"""
filtered_df = filtered_df[filtered_df["ser_year"] != 2011]
filtered_df = filtered_df[filtered_df["ser_stklf"] != 2]

In [316]:
# drop companies w/ growth rates that are NaN
filtered_df = filtered_df.dropna(subset=["TURNOVER_GROWTH"])

# drop companies w/ growth rates that are exactly 0. For now I don't know how to handle these or where they stem from.
filtered_df = filtered_df[filtered_df["TURNOVER_GROWTH"] != 0]

In [318]:
filtered_df["HIGH_GROWTH"].value_counts()

HIGH_GROWTH
0    87117
1     2738
Name: count, dtype: int64

# Industry Variable

In [319]:
"""
There are a lot of columns in the dataset that is about the comapnies industry, a lot of different SNI codes.
But we've decided to use the 'bransch_borsbransch_konv' column as the industry variable. Which is a conversion of the SNI codes
to fewer branches.

Some companies have changed industry over time, so we determine the most frequent industry for each company and add it as a new column
to each row assicated with that company.
"""

# add one industry to all companies.
dict_of_industries = {
    10: 'Energy & Environment',
    15: 'Materials',
    20: 'Industrial goods',
    22: 'Construction industry',
    25: 'Shopping goods',
    30: 'Convenience goods',
    35: 'Health & Education',
    40: 'Finance & Real estate',
    45: 'IT & Electronics',
    50: 'Telecom & Media',
    60: 'Corporate services',
    98: 'Other',
    99: 'SNI07 missing'
}

# Determine the most frequent 'bransch_borsbransch_konv' value for each company
most_frequent_industry = filtered_df.groupby('ORGNR')['bransch_borsbransch_konv'].agg(lambda x: x.mode()[0])

# Map the most frequent 'bransch_borsbransch_konv' value to the corresponding industry name
most_frequent_industry = most_frequent_industry.map(dict_of_industries)

# Add the new 'INDUSTRY' column to the DataFrame
filtered_df = filtered_df.merge(most_frequent_industry.rename('INDUSTRY'), on='ORGNR')

# Filter out rows where the 'INDUSTRY' column is 'SNI07 missing' or 'Other'
filtered_df = filtered_df[~filtered_df['INDUSTRY'].isin(['SNI07 missing', 'Other'])]

# Variables

In [320]:
len(filtered_df)

88456

In [321]:
"""
Some more filtering to remove weird values.

Negative assets, negative debt etc.

It removes about 150 observations.
"""

# drop observations with no assets
filtered_df = filtered_df[filtered_df["br09_tillgsu"] > 0]

# drop observations with negative tangible assets
filtered_df = filtered_df[filtered_df["br02_matanlsu"] >= 0]

# drop observations with negative current assets
filtered_df = filtered_df[filtered_df["br08_omstgsu"] > 0]

# drop observations with negative inventories
filtered_df = filtered_df[filtered_df["br06c_lagersu"] >= 0]

# drop observations with positive depreciation
filtered_df = filtered_df[filtered_df["rr05_avskriv"] <= 0]

# drop one weird outlier when it comes to interest expenses
filtered_df = filtered_df[filtered_df["rr09_finkostn"] > -200000]

# drop observations with negative long-term debt
filtered_df = filtered_df[filtered_df["br15_lsksu"] >= 0]

# drop observations with negative short-term debt
filtered_df = filtered_df[filtered_df["br13_ksksu"] > 0]

In [322]:
len(filtered_df)

88325

## Independent

In [323]:
"""
Firm size

Some articles use the log of sales, others the log of assets. We'll add both to the dataset and see which one works best.
"""

filtered_df["SIZE_LOG_ASSETS"] = np.log(filtered_df["br09_tillgsu"])
filtered_df["SIZE_LOG_SALES"] = np.log(filtered_df["rr01_ntoms"])

In [327]:
"""
Asset tangibility

Most common thing to add as a proxy is the ratio of tangible fixed assets to total assets.
Maybe include intangible assets as well, are such assets (patents, brands etc) used as collateral?

There are quite a lot of companies with 0 tangible assets (8000 observations), which is a bit strange. Maybe we should remove those.
"""

# tangible assets / total assets
filtered_df["ASSET_TANGIBILITY"] = filtered_df["br02_matanlsu"] / filtered_df["br09_tillgsu"]

In [330]:
"""
liquidity on debt

Cole use cash and cash equivalents to total assets, but we could also use current ratio or quick ratio.
"""

# Liquid assets (cash and market securities) to total assets
filtered_df["LIQUIDITY"] = filtered_df["br07_kplackaba"] / filtered_df["br09_tillgsu"]

# Current ratio: current assets divided by current liabilities
filtered_df["CURRENT_RATIO"] = filtered_df["br08_omstgsu"] / filtered_df["br13_ksksu"]

# Quick ratio: (current assets - inventory) divided by current liabilities
filtered_df["QUICK_RATIO"] = (filtered_df["br08_omstgsu"] - filtered_df["br06c_lagersu"]) / filtered_df["br13_ksksu"]


In [361]:
"""
Profitability

There are many measures for this, RoA, RoE, operating margin, net margin etc.
"""

# Profit margin according to serrano is already defined as 'ny_vinstprc'

filtered_df['ny_vinstprc'].value_counts(dropna=False)

ny_vinstprc
0.000000    63
0.062500     6
0.038462     4
0.052632     4
0.064516     4
            ..
0.046418     1
0.023444     1
0.049516     1
0.009657     1
0.022095     1
Name: count, Length: 87969, dtype: int64

In [335]:
"""
Tax shields

For now I only find definition in Vanacker & Manigart (2010).
"""

# Debt tax shields: Interest expenses / Total Assets
filtered_df['DEBT_TAX_SHIELDS'] = filtered_df['rr09_finkostn'] / filtered_df['br09_tillgsu']

# Non-debt tax shields: Depreciation and amortization / Total Assets
filtered_df['NON_DEBT_TAX_SHIELDS'] = filtered_df['rr05_avskriv'] / filtered_df['br09_tillgsu']

In [252]:
"""
Tax

Do we mean the corporate tax in Sweden or the % tax the companies actually pay?
"""

'\nTax\n\nDo we mean the corporate tax in Sweden or the % tax the companies actually pay?\n'

In [ ]:
"""
Growth opportunities

Some have change in sales too. But that gets weird for us since that is what we use to determine HGFs.
"""

# % change in total assets
filtered_df['GROWTH_OPPORTUNITIES_ASSETS'] = filtered_df.groupby('ORGNR')['br09_tillgsu'].pct_change()


In [ ]:
filtered_df["GROWTH_OPPORTUNITIES_ASSETS"].value_counts(dropna=False)
# I don't know how there can be NaNs.

GROWTH_OPPORTUNITIES_ASSETS
 NaN         23593
 0.000000      225
 0.200000        7
 0.142857        5
 0.166667        5
             ...  
-0.075483        1
 0.041977        1
-0.140195        1
-0.007380        1
-0.278154        1
Name: count, Length: 64131, dtype: int64

In [342]:
"""
Age

Need to discuss this since age could be year incorporated vs todays date, or 2019, or for every year of observation
for one company.
"""

# Change reg date to only year
filtered_df["ser_regdat"] = pd.to_datetime(filtered_df["ser_regdat"])
filtered_df["ser_regdat"] = filtered_df["ser_regdat"].dt.year

# Age of the company (year of observation - year of registration)
filtered_df["AGE"] = filtered_df["ser_year"] - filtered_df["ser_regdat"]

## Dependent

In [345]:
"""
Leverage
"""

# Total debt to total assets
filtered_df['LEVERAGE_TOTAL'] = (filtered_df['br15_lsksu'] + filtered_df['br13_ksksu']) / filtered_df['br09_tillgsu']

# Long-term debt to total assets (all non-current liabilities, could be other things)
filtered_df['LEVERAGE_LONG'] = filtered_df['br15_lsksu'] / filtered_df['br09_tillgsu']

# Short-term debt to total assets (all current liabilities, could be other things)
filtered_df['LEVERAGE_SHORT'] = filtered_df['br13_ksksu'] / filtered_df['br09_tillgsu']

# Misc.

In [ ]:
# save to a excel file
filtered_df.head(10000).to_excel("filtered_df.xlsx", index=False)

# cheking stuff
filtered_df["br15_lsksu"].value_counts(dropna=False)
# see if any company have negative br09_tillgsu
filtered_df[filtered_df["br15_lsksu"] < 0]
#filtered_df["rr09_finkostn"].describe()

# Split the dataset in HGF vs non-HGF

In [349]:
# Identify HGF ORGNR
high_growth_ids = filtered_df[filtered_df["HIGH_GROWTH"] == 1]["ORGNR"].unique()

# Create a new DataFrame with only high-growth firms
HGF_df = filtered_df[filtered_df["ORGNR"].isin(high_growth_ids)]

# Create a new DataFrame with only non-high-growth firms
non_HGF_df = filtered_df[~filtered_df["ORGNR"].isin(high_growth_ids)]

In [350]:
display(len(HGF_df), len(non_HGF_df))
display(HGF_df["ORGNR"].nunique(), non_HGF_df["ORGNR"].nunique())

8222

80103

1937

21656

# Descriptive Data

In [351]:
"""
Industry
"""
# Get one row per company (HGF)
unique_HGFs = HGF_df.drop_duplicates(subset=["ORGNR"])

# Get one row per company (non_HGF)
unique_non_HGFs = non_HGF_df.drop_duplicates(subset=["ORGNR"])

display(unique_HGFs["INDUSTRY"].value_counts(normalize=True) * 100)
display(unique_non_HGFs["INDUSTRY"].value_counts(normalize=True) * 100)

INDUSTRY
Corporate services       27.465152
Construction industry    27.258647
Shopping goods           14.455343
Health & Education       11.099639
IT & Electronics          7.434177
Industrial goods          4.594734
Convenience goods         2.891069
Finance & Real estate     2.271554
Materials                 1.135777
Telecom & Media           0.929272
Energy & Environment      0.464636
Name: proportion, dtype: float64

INDUSTRY
Corporate services       23.623938
Shopping goods           22.109346
Construction industry    21.596786
Industrial goods          8.417990
Health & Education        8.256372
Convenience goods         7.586812
IT & Electronics          3.338567
Finance & Real estate     1.823975
Materials                 1.703916
Telecom & Media           0.992796
Energy & Environment      0.549501
Name: proportion, dtype: float64

In [352]:
"""
Leverage
"""
display(HGF_df["LEVERAGE_SHORT"].describe())
display(non_HGF_df["LEVERAGE_SHORT"].describe())
display(HGF_df["LEVERAGE_LONG"].describe())
display(non_HGF_df["LEVERAGE_LONG"].describe())
display(HGF_df["LEVERAGE_TOTAL"].describe())
display(non_HGF_df["LEVERAGE_TOTAL"].describe())

count    8222.000000
mean        0.562633
std         0.419246
min         0.023494
25%         0.390730
50%         0.541047
75%         0.699702
max        31.113208
Name: LEVERAGE_SHORT, dtype: float64

count    80103.000000
mean         0.546419
std         11.983994
min          0.000767
25%          0.316832
50%          0.464484
75%          0.639714
max       3390.000000
Name: LEVERAGE_SHORT, dtype: float64

count    8222.000000
mean        0.111813
std         0.171862
min         0.000000
25%         0.000000
50%         0.016527
75%         0.175766
max         2.289017
Name: LEVERAGE_LONG, dtype: float64

count    80103.000000
mean         0.129868
std          0.374154
min          0.000000
25%          0.000000
50%          0.025869
75%          0.211284
max         47.798479
Name: LEVERAGE_LONG, dtype: float64

count    8222.000000
mean        0.674446
std         0.424567
min         0.046232
25%         0.504874
50%         0.674295
75%         0.822678
max        31.113208
Name: LEVERAGE_TOTAL, dtype: float64

count    80103.000000
mean         0.676287
std         11.994153
min          0.002862
25%          0.439318
50%          0.613147
75%          0.784030
max       3390.000000
Name: LEVERAGE_TOTAL, dtype: float64

In [353]:
"""
Firm size
"""
display(HGF_df["SIZE_LOG_ASSETS"].describe())
display(non_HGF_df["SIZE_LOG_ASSETS"].describe())
display(HGF_df["SIZE_LOG_SALES"].describe())
display(non_HGF_df["SIZE_LOG_SALES"].describe())

count    8222.000000
mean        8.904761
std         1.040719
min         3.970292
25%         8.196161
50%         8.851449
75%         9.557558
max        14.915306
Name: SIZE_LOG_ASSETS, dtype: float64

count    80103.000000
mean         8.949728
std          1.082898
min          0.000000
25%          8.236685
50%          8.937744
75%          9.623641
max         16.914990
Name: SIZE_LOG_ASSETS, dtype: float64

count    8222.000000
mean        9.867311
std         0.869937
min         3.401197
25%         9.320651
50%         9.808022
75%        10.366537
max        14.877591
Name: SIZE_LOG_SALES, dtype: float64

count    80103.000000
mean         9.867097
std          0.890971
min          0.000000
25%          9.304377
50%          9.773265
75%         10.361766
max         15.618990
Name: SIZE_LOG_SALES, dtype: float64

In [354]:
"""
Asset tangibility
"""
display(HGF_df["ASSET_TANGIBILITY"].describe())
display(non_HGF_df["ASSET_TANGIBILITY"].describe())

count    8222.000000
mean        0.178163
std         0.230602
min         0.000000
25%         0.011363
50%         0.062971
75%         0.275648
max         0.965418
Name: ASSET_TANGIBILITY, dtype: float64

count    80103.000000
mean         0.226686
std          0.248352
min          0.000000
25%          0.022975
50%          0.117036
75%          0.385131
max          0.996224
Name: ASSET_TANGIBILITY, dtype: float64

In [355]:
"""
liquidity
"""
display(HGF_df["LIQUIDITY"].describe())
display(non_HGF_df["LIQUIDITY"].describe())
display(HGF_df["CURRENT_RATIO"].describe())
display(non_HGF_df["CURRENT_RATIO"].describe())
display(HGF_df["QUICK_RATIO"].describe())
display(non_HGF_df["QUICK_RATIO"].describe())

count    8222.000000
mean        0.258230
std         0.226480
min        -0.166153
25%         0.059602
50%         0.208982
75%         0.409515
max         1.128364
Name: LIQUIDITY, dtype: float64

count    80103.000000
mean         0.250082
std          0.225441
min         -0.464189
25%          0.053498
50%          0.198467
75%          0.394084
max          1.760000
Name: LIQUIDITY, dtype: float64

count    8222.000000
mean        1.612631
std         1.050302
min         0.032141
25%         1.064974
50%         1.390331
75%         1.874925
max        19.233420
Name: CURRENT_RATIO, dtype: float64

count    80103.000000
mean         1.842341
std          6.429229
min          0.000295
25%          1.075794
50%          1.474479
75%          2.068219
max       1303.666667
Name: CURRENT_RATIO, dtype: float64

count    8222.000000
mean        1.492987
std         1.042963
min         0.031549
25%         0.938966
50%         1.293448
75%         1.765038
max        19.233420
Name: QUICK_RATIO, dtype: float64

count    80103.000000
mean         1.567765
std          6.350414
min         -2.636364
25%          0.840593
50%          1.255836
75%          1.810539
max       1303.666667
Name: QUICK_RATIO, dtype: float64

In [356]:
"""
Profitability
"""
display(HGF_df["ny_vinstprc"].describe())
display(non_HGF_df["ny_vinstprc"].describe())

count    8222.000000
mean       -0.098134
std         6.337403
min      -509.500000
25%         0.015382
50%         0.049869
75%         0.104954
max        12.133333
Name: ny_vinstprc, dtype: float64

count    80103.000000
mean        -0.508930
std         73.215937
min     -12026.000000
25%          0.010176
50%          0.039234
75%          0.084947
max       2959.000000
Name: ny_vinstprc, dtype: float64

In [357]:
"""
Tax shields
"""
display(HGF_df["DEBT_TAX_SHIELDS"].describe())
display(non_HGF_df["DEBT_TAX_SHIELDS"].describe())
display(HGF_df["NON_DEBT_TAX_SHIELDS"].describe())
display(non_HGF_df["NON_DEBT_TAX_SHIELDS"].describe())

count    8222.000000
mean       -0.010378
std         0.029588
min        -0.932665
25%        -0.012181
50%        -0.004237
75%        -0.000779
max         0.000000
Name: DEBT_TAX_SHIELDS, dtype: float64

count    80103.000000
mean        -0.014487
std          0.901509
min       -255.000000
25%         -0.013719
50%         -0.004895
75%         -0.000840
max          0.000000
Name: DEBT_TAX_SHIELDS, dtype: float64

count    8222.000000
mean       -0.042445
std         0.060741
min        -2.184781
25%        -0.060804
50%        -0.020802
75%        -0.005337
max         0.000000
Name: NON_DEBT_TAX_SHIELDS, dtype: float64

count    80103.000000
mean        -0.053929
std          0.646231
min       -175.000000
25%         -0.072632
50%         -0.029436
75%         -0.009339
max          0.000000
Name: NON_DEBT_TAX_SHIELDS, dtype: float64

In [358]:
"""
Growth opportunities
"""
display(HGF_df["GROWTH_OPPORTUNITIES_ASSETS"].describe())
display(non_HGF_df["GROWTH_OPPORTUNITIES_ASSETS"].describe())

count    6285.000000
mean        0.293369
std         0.577566
min        -0.941886
25%         0.025044
50%         0.198993
75%         0.423978
max        14.164602
Name: GROWTH_OPPORTUNITIES_ASSETS, dtype: float64

count    58447.000000
mean         0.075705
std          0.363814
min         -0.999795
25%         -0.058877
50%          0.036513
75%          0.153152
max         43.918590
Name: GROWTH_OPPORTUNITIES_ASSETS, dtype: float64

In [359]:
"""
Age
"""
display(HGF_df["AGE"].describe())
display(non_HGF_df["AGE"].describe())

count    8222.000000
mean        9.313063
std         8.203412
min         1.000000
25%         4.000000
50%         7.000000
75%        11.000000
max        60.000000
Name: AGE, dtype: float64

count    80103.000000
mean        17.544886
std         14.058056
min          1.000000
25%          7.000000
50%         14.000000
75%         25.000000
max        152.000000
Name: AGE, dtype: float64

# Tests